In [2]:
import os 
from dotenv import load_dotenv
load_dotenv('C:/Users/midon/Documents/anomaly-detection-autoencoder-based-basic/.env.local')
PROJECT_PATH = os.getenv('PROJECT_PATH')
os.chdir(PROJECT_PATH)

import pandas as pd
import numpy as np
from preprocessing.preprocessing import * 
from eval.evaluate_outliers import *
from visualisation import * 
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from models.models import Generator, Discriminator

In [ ]:

def generate_fake_data(experiment_name, n = 1e+5):
    base_path = 'saved_models'
    path = os.path.join(base_path, experiment_name)
    if not os.path.exists(path):
        raise FileNotFoundError(f"Experiment path not found: {path}")
    with open('synthetic_data/data/transformer.pkl', 'rb') as f:
        transformer = pickle.load(f)
    with open(os.path.join(path, 'hyperparams.pkl'),'rb') as r:
        hyperparams = pickle.load(r)
    n_binary_cols = len(transformer.get_OHEncoded_cols()) + len(transformer.bool_cols)
    input_dim = len(transformer.input_cols)

    generator_model_state = torch.load(os.path.join(path, "generator_weights.pth"))
    generator = Generator(n_binary_cols = n_binary_cols, noise_dim = hyperparams['noise_dim'], output_dim=input_dim, hidden_dim = hyperparams['hidden_dim_g'])
    generator.load_state_dict(generator_model_state)
    generator.eval()
    noise = torch.randn(len(n), hyperparams['noise_dim'])
    with torch.no_grad():
        fake_data = generator(noise)
    return fake_data


In [ ]:
base_path = 'saved_models'
experiment_name = 'first_gan'
path = os.path.join(base_path, experiment_name)

if not os.path.exists(path):
    raise FileNotFoundError(f"Experiment path not found: {path}")

generator_model_state = torch.load(os.path.join(path, "generator_weights.pth"))
discriminator_model_state = torch.load(os.path.join(path, "discriminator_weights.pth"))
# model = torch.load(os.path.join(path, "full_model.pth"), weights_only=False)
# with open(os.path.join(path, 'epoch_losses.pkl'), 'rb') as f:
#     epoch_losses = pickle.load(f)
with open('synthetic_data/data/train_test_data.pkl', 'rb') as f:
        loaded_data = pickle.load(f)
        
with open('synthetic_data/data/transformer.pkl', 'rb') as f:
    transformer = pickle.load(f)

with open(os.path.join(path, 'hyperparams.pkl'),'rb') as r:
    hyperparams = pickle.load(r)

X_train_df = loaded_data['X_train']
X_test_df = loaded_data['X_test']
y_train = np.array(loaded_data['y_train'])
y_test = np.array(loaded_data['y_test'])
X_train = transformer.transform_input()
X_test = transformer.transform_input_X(loaded_data['X_test'])
input_dim = len(transformer.input_cols)
n_binary_cols = len(transformer.get_OHEncoded_cols()) + len(transformer.bool_cols)
attributes_dict = transformer.get_metadata() 

generator = Generator(n_binary_cols = n_binary_cols, noise_dim = hyperparams['noise_dim'], output_dim=input_dim, hidden_dim = hyperparams['hidden_dim_g'])
generator.load_state_dict(generator_model_state)
generator.eval()

Generator(
  (network): Sequential(
    (0): Linear(in_features=100, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
    (3): Linear(in_features=256, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.2)
    (6): Linear(in_features=512, out_features=512, bias=True)
    (7): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.2)
    (9): Linear(in_features=512, out_features=256, bias=True)
    (10): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.2)
    (12): Linear(in_features=256, out_features=20, bias=True)
  )
)

In [16]:
torch.randn(100000, hyperparams['noise_dim'])

tensor([[ 1.4980e-01,  8.5583e-02, -3.7402e-01,  ...,  6.9930e-01,
          1.5494e-01,  4.9579e-01],
        [-1.9593e+00, -1.8358e-02,  2.6771e+00,  ..., -8.2552e-01,
         -7.6169e-01, -6.6257e-02],
        [ 1.4604e+00, -1.7068e-01,  4.7000e-01,  ...,  1.4868e+00,
          1.0391e+00, -1.4763e-02],
        ...,
        [-1.1207e-03,  1.1995e-01,  5.5211e-01,  ...,  6.7939e-01,
         -5.8932e-01, -2.9709e-01],
        [-1.5969e-01, -3.4899e-01, -3.8697e-01,  ...,  1.1886e+00,
          1.9711e+00,  3.7213e-02],
        [ 1.4650e+00,  3.5822e-01,  1.0872e+00,  ..., -1.3321e+00,
          1.3445e+00,  1.0919e+00]])

In [7]:
normal_train_data = X_train[y_train == 0]
noise = torch.randn(len(normal_train_data), hyperparams['noise_dim'])
with torch.no_grad():
    fake_data = generator(noise)
fake_data.detach().numpy()

array([[ 2.18246202e-03,  9.94422473e-04,  9.99123871e-01, ...,
        -1.47809274e-04,  2.54687537e-02,  4.68899578e-01],
       [ 6.38693891e-05,  1.57717091e-04,  8.53627384e-01, ...,
        -5.19179739e-04, -1.17973611e-03, -1.19483471e-02],
       [ 7.25446516e-05,  1.93226995e-04,  3.23907938e-03, ...,
        -1.64530613e-02, -2.75267288e-03, -1.02018625e-01],
       ...,
       [ 6.97822106e-05,  1.28346874e-04,  2.57816585e-03, ...,
        -1.19169056e-02, -2.39612814e-03, -9.99232978e-02],
       [ 6.28625785e-05,  9.23530315e-05,  2.04014778e-03, ...,
        -8.87374580e-03, -4.33235429e-04, -1.08552895e-01],
       [ 2.02341255e-07,  6.10076313e-06,  2.99028873e-01, ...,
        -5.75750992e-02, -4.67152819e-02, -1.04293868e-01]],
      shape=(146066, 20), dtype=float32)

In [12]:
fake_df = transformer.transform_preds(fake_data.detach().numpy())
fake_df

,DISBURSED_AMOUNT,LTV,VOTERID_FLAG,PERFORM_CNS_SCORE_DESCRIPTION,PRI_CURRENT_BALANCE,SEC_CURRENT_BALANCE,SEC_INSTAL_AMT,NEW_ACCTS_IN_LAST_SIX_MONTHS,NO_OF_INQUIRIES,AGE_DISBURSMENT,SALARIED_FLAG,PRI_SANCTION_GAP,SEC_SANCTION_GAP,LTA_LTV,PRI_EMI_DISBURSED_RATIO,SEC_EMI_DISBURSED_RATIO,OUT_BALANCE_DISBURSED_RATIO
0,63018,77.180428,1,Insufficient / Non-standard Data,46195,-5211,0,0,0,26,0,3474,-179,0.972930,1414.315308,76.349976,5857.964844
1,53524,74.181030,0,Insufficient / Non-standard Data,34099,3798,0,0,0,29,0,587,-120,0.970363,1385.996216,32.722103,1144.336060
2,52325,75.818481,0,Low Risk,171269,6659,0,0,0,32,0,2556,-132,0.969822,170.944336,30.146950,261.400116
3,51298,73.733421,0,Insufficient / Non-standard Data,249448,-3230,0,0,0,39,1,8704,-306,0.963822,0.000000,0.000000,196.829071
4,51254,73.898552,0,Low Risk,108464,8018,0,0,0,32,0,901,-63,0.969785,820.904602,35.917503,244.111603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146061,53906,81.800919,0,Insufficient / Non-standard Data,150878,1878,0,0,0,33,0,1250,-693,0.969344,0.000000,0.000000,126.965538
146062,56409,76.074524,0,Insufficient / Non-standard Data,33772,735,0,0,0,27,0,922,-222,0.970584,1459.318726,51.104733,2459.044922
146063,51823,74.947540,0,Low Risk,146491,7568,0,0,0,32,0,1725,-79,0.969954,516.852783,30.730671,281.940094
146064,51116,73.649490,0,Low Risk,118483,7906,0,0,0,32,0,1050,-35,0.969739,748.911560,33.944244,197.346405


In [10]:
transformer.transform_preds(normal_train_data)

,DISBURSED_AMOUNT,LTV,VOTERID_FLAG,PERFORM_CNS_SCORE_DESCRIPTION,PRI_CURRENT_BALANCE,SEC_CURRENT_BALANCE,SEC_INSTAL_AMT,NEW_ACCTS_IN_LAST_SIX_MONTHS,NO_OF_INQUIRIES,AGE_DISBURSMENT,SALARIED_FLAG,PRI_SANCTION_GAP,SEC_SANCTION_GAP,LTA_LTV,PRI_EMI_DISBURSED_RATIO,SEC_EMI_DISBURSED_RATIO,OUT_BALANCE_DISBURSED_RATIO
0,57462,83.542404,0,Low Risk,218133,4375,0,3,2,28,1,59629,0,0.962536,0.166504,0.000001,1.761803
1,63168,73.562508,0,Low Risk,127062,1643,0,1,0,30,0,14141,0,0.976304,0.082031,0.000001,1.082848
2,52583,87.339325,0,Insufficient / Non-standard Data,10010,4813,62,0,0,45,1,0,-449,0.975887,0.000000,0.023465,5.610314
3,54145,84.797363,0,Insufficient / Non-standard Data,29006,2115,0,0,0,27,1,0,-121,0.967692,1600.000000,0.000001,56.883507
4,53308,78.645355,0,Insufficient / Non-standard Data,12876,84,5,0,1,26,0,3689,-580,0.955858,0.000000,0.006143,15.919395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146061,33621,76.187202,0,Insufficient / Non-standard Data,15539,1452,0,0,0,26,0,-17790,0,0.983167,0.000000,0.000001,0.955040
146062,96427,72.997101,1,Insufficient / Non-standard Data,35171,-1126,129,0,0,23,0,0,-3,0.974620,0.000000,0.054521,14.388756
146063,43030,77.152786,1,Insufficient / Non-standard Data,20205,1012,0,0,0,33,0,0,-399,0.970209,0.000000,0.000001,10.141563
146064,51219,75.898270,0,Low Risk,28162,5589,0,0,0,34,0,12505,0,0.969399,0.072143,0.000001,2.733483
